In [35]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from statsmodels.sandbox.stats.multicomp import multipletests
from collections import OrderedDict

In [50]:
path='../Result_trip/'
ctrl1=pd.read_csv(path+'ctrl1_fitting_aggr.csv',low_memory=False)
ctrl2=pd.read_csv(path+'ctrl2_fitting_aggr.csv',low_memory=False)
ctrl3=pd.read_csv(path+'ctrl3_fitting_aggr.csv',low_memory=False)
treated1=pd.read_csv(path+'treated1_fitting_aggr.csv',low_memory=False)
treated2=pd.read_csv(path+'treated2_fitting_aggr.csv',low_memory=False)
treated3=pd.read_csv(path+'treated3_fitting_aggr.csv',low_memory=False)

Pgroups=pd.read_csv(path+'Pgroups.csv')

ctrl1.loc[ctrl1[(ctrl1['ctrl1_R']<0.8) | (ctrl1['ctrl1_Pl']>0.3)].index,ctrl1.columns[4:]]=float('nan')
ctrl2.loc[ctrl2[(ctrl2['ctrl2_R']<0.8) | (ctrl2['ctrl2_Pl']>0.3)].index,ctrl2.columns[4:]]=float('nan')
ctrl3.loc[ctrl3[(ctrl3['ctrl3_R']<0.8) | (ctrl3['ctrl3_Pl']>0.3)].index,ctrl3.columns[4:]]=float('nan')

treated1.loc[treated1[treated1['treated1_R']<0.8].index,treated1.columns[4:]]=float('nan')
treated2.loc[treated2[treated2['treated2_R']<0.8].index,treated2.columns[4:]]=float('nan')
treated3.loc[treated3[treated3['treated3_R']<0.8].index,treated3.columns[4:]]=float('nan')

In [51]:
diff_rep1 = pd.DataFrame(index=range(0,ctrl1.shape[0]))
diff_rep1['diff_rep1'] = treated1['treated1_min']-ctrl1['ctrl1_min']
diff_rep1['slope_rep1'] = np.fmin(ctrl1['ctrl1_slope'],treated1['treated1_slope'])
diff_rep2 = pd.DataFrame(index=range(0,ctrl2.shape[0]))
diff_rep2['diff_rep2'] = treated2['treated2_min']-ctrl2['ctrl2_min']
diff_rep2['slope_rep2'] = np.fmin(ctrl2['ctrl2_slope'],treated2['treated2_slope'])
diff_rep3 = pd.DataFrame(index=range(0,ctrl3.shape[0]))
diff_rep3['diff_rep3'] = treated3['treated3_min']-ctrl3['ctrl3_min']
diff_rep3['slope_rep3'] = np.minimum(ctrl3['ctrl3_slope'],treated3['treated3_slope'])

In [52]:
def correct_fdr(p_values):
   p_values= {k:v for k,v in enumerate(p_values)}
   n_comparison= len(p_values)
   modifier= n_comparison

   indexed_p_values= OrderedDict({(j[1],i):j[0] for i, j in enumerate(p_values.items())})

   indexed_p_values= OrderedDict(sorted(indexed_p_values.items(), reverse=True))

   for (value, i),ind in indexed_p_values.items():
       new_value= p_values[ind] * n_comparison / float(modifier)

       p_values[ind]= new_value if new_value <=1 else 1 
       modifier -= 1 

   fdr= [1]*len(p_values)

   for i,value in p_values.iteritems():
       fdr[i]= value 

   return fdr
def combine_col(df,string):
    df1=df[df.columns[0:13]]
    df1['mp_'+string]=df[string+'_min'].tolist()
    df1[string+'_slope']=df[string+'_slope'].tolist()
    return df1
def diff_rep(df1,string,groupname):
    use_me=list(df1['slope_' + string].dropna().index)
    slope_rep1=df1.ix[use_me]['slope_' + string].sort_values(ascending=False)
    slope_order=slope_rep1.index
    diff_re=df1.ix[slope_order]['diff_' + string]
    diff_quant = []
    diff_z = [float('nan')]*len(diff_re)
    bin_size=300
    for i in range(0,int(np.floor(len(diff_re)/bin_size))):
        if len(diff_re) - bin_size*(i+1) < bin_size:
            y = len(diff_re)
        else:
            y = bin_size * (i+1)
        l = diff_re.iloc[(bin_size*(i)): y].quantile([.1587, .50, .8413])
        for j in range((bin_size*(i)),y):
            if (np.isnan(diff_re.iloc[j])):
                diff_z[j] =float('nan')
            elif (diff_re.iloc[j] < l.ix[.50]):
                diff_z[j] = (l.ix[.50] - diff_re.iloc[j])/(l.ix[.50]-l.ix[.1587])
            else:
                diff_z[j] = (diff_re.iloc[j]-l.ix[.50])/(l.ix[.8413]-l.ix[.50])
        diff_quant.append(l.tolist())
    diff_z_abs=map(abs,diff_z)
    diff_prob=1-norm.cdf(np.abs(diff_z))
    diff_prob1=[i for i in range(0,len(diff_prob)) if not(np.isnan(diff_prob[i]))]
    diff_prob2=[diff_prob[i] for i in diff_prob1]
    diff_prob2=multipletests(diff_prob2,method='fdr_bh')[1]
    for i in range(0,len(diff_prob1)):
        diff_prob[diff_prob1[i]]=diff_prob2[i]
    groupname=groupname.reset_index(drop=True)
    names=groupname.ix[use_me]
    df2 = pd.concat([names.ix[slope_order],df1.ix[slope_order]['diff_'+string]],1)
    df2['diff_z_'+string]=diff_z
    df2['diff_prob_'+string]=diff_prob
    df2= pd.concat([df2,slope_rep1.ix[slope_order]],1)
    return df2

def diff_repAll(df1,string,groupname):
    use_me=list(df1['slope_' + string].dropna().index)
    slope_rep1=df1.ix[use_me]['slope_' + string].sort_values(ascending=False)
    slope_order=slope_rep1.index
    diff_re=df1.ix[slope_order]['diff_' + string]
    diff_quant = []
    diff_z = [float('nan')]*len(diff_re)
    l = diff_re.quantile([.1587, .50, .8413])
    for j in range(0,len(diff_re)):
        if (np.isnan(diff_re.iloc[j])):
            diff_z[j] =float('nan')
        elif (diff_re.iloc[j] < l.ix[.50]):
            diff_z[j] = (l.ix[.50] - diff_re.iloc[j])/(l.ix[.50]-l.ix[.1587])
        else:
            diff_z[j] = (diff_re.iloc[j]-l.ix[.50])/(l.ix[.8413]-l.ix[.50])
    diff_quant.append(l.tolist())
    diff_z_abs=map(abs,diff_z)
    diff_prob=1-norm.cdf(np.abs(diff_z))
    diff_prob1=[i for i in range(0,len(diff_prob)) if not(np.isnan(diff_prob[i]))]
    diff_prob2=[diff_prob[i] for i in diff_prob1]
    diff_prob2=multipletests(diff_prob2,method='fdr_bh')[1]
    for i in range(0,len(diff_prob1)):
        diff_prob[diff_prob1[i]]=diff_prob2[i]
    groupname=groupname.reset_index(drop=True)
    names=groupname.ix[use_me]
    df2 = pd.concat([names.ix[slope_order],df1.ix[slope_order]['diff_'+string]],1)
    df2['diff_z_'+string]=diff_z
    df2['diff_prob_'+string]=diff_prob
    df2= pd.concat([df2,slope_rep1.ix[slope_order]],1)
    return slope_order

In [53]:
diff_rep1=diff_rep(diff_rep1,'rep1',Pgroups)
diff_rep2=diff_rep(diff_rep2,'rep2',Pgroups)
diff_rep3=diff_rep(diff_rep3,'rep3',Pgroups)

In [40]:
ctrl1fit=combine_col(ctrl1,'ctrl1')
ctrl2fit=combine_col(ctrl2,'ctrl2')
ctrl3fit=combine_col(ctrl3,'ctrl3')
treated1fit=combine_col(treated1,'treated1')
treated2fit=combine_col(treated2,'treated2')
treated3fit=combine_col(treated3,'treated3')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
# result=ctrl1fit.merge(treated1fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(ctrl2fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
# .merge(treated2fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(diff_rep1,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
# .merge(diff_rep2,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])

result=ctrl1fit.merge(treated1fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(ctrl2fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
.merge(treated2fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(ctrl3fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
.merge(treated3fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(diff_rep1,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
.merge(diff_rep2,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(diff_rep3,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])

result.to_csv(path+'peptides_fit_aggr.csv',index=False)

In [42]:
same_thermal_shift_direction = (((result['diff_rep1'] < 0) & (result['diff_rep2'] < 0)) & (result['diff_rep3'] < 0)) | (((result['diff_rep1'] > 0) & (result['diff_rep2'] > 0)) & (result['diff_rep3'] > 0))
slope_steep_enough = (result['slope_rep1'] < -0.06) | (result['slope_rep2'] < -0.06) | (result['slope_rep2'] < -0.06)
thermal_shift_greater_than_difference_vehicle = np.fmin(np.fmin(map(abs,result['diff_rep1']),map(abs,result['diff_rep2'])),map(abs,result['diff_rep2'])) > np.fmax(np.fmax((map(abs,result['mp_ctrl1']-result['mp_ctrl2'])),(map(abs,result['mp_ctrl1']-result['mp_ctrl3']))),(map(abs,result['mp_ctrl3']-result['mp_ctrl2'])))
pval=0.05
x = result["diff_prob_rep1"] < pval
y = result["diff_prob_rep2"] < pval
z = result["diff_prob_rep3"] < pval
pvalue_criteria3 = ((x == True) & (y == True)) &  (z==True)
pvalue_criteria2 = ((x == True) & (y == True)) |  ((x == True) & (z == True)) |  ((y == True) & (z == True))
fulfills_all =  (same_thermal_shift_direction==True)  & (thermal_shift_greater_than_difference_vehicle == True) & (pvalue_criteria2==True) & (slope_steep_enough==True)
in_both_replicates = ((~np.isnan(result["diff_rep1"])) & (~np.isnan(result["diff_rep2"]))) & (~np.isnan(result["diff_rep3"]))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in greater
  app.launch_new_instance()


In [43]:
criteria=pd.concat([same_thermal_shift_direction, slope_steep_enough, in_both_replicates,pd.Series(thermal_shift_greater_than_difference_vehicle), pvalue_criteria2, pvalue_criteria3,fulfills_all],1)
criteria=criteria.fillna(False)
criteria.columns=['same_thermal_shift_direction', 'slope_steep_enough', 'in_both_replicates','thermal_shift_greater_than_difference_vehicle', 'pvalue_criteria2','pvalue_criteria3','fulfills_all']
result=pd.concat([result,criteria],1)

In [63]:
res_temp=['O43865','Q9BYT8','P08238','Q9HCG8','Q58FF6','Q00796']

In [65]:
result[(~(result['fulfills_all']==True))&(result['Protein Group Accessions'].isin(res_temp))]

,Protein Group Accessions,Protein_Description,Gene_Name,ctrl1_37,ctrl1_40,ctrl1_43,ctrl1_46,ctrl1_49,ctrl1_52,ctrl1_55,...,diff_z_rep3,diff_prob_rep3,slope_rep3,same_thermal_shift_direction,slope_steep_enough,in_both_replicates,thermal_shift_greater_than_difference_vehicle,pvalue_criteria2,pvalue_criteria3,fulfills_all


In [62]:
# result[result['fulfills_all']==True][['Protein Group Accessions','mp_ctrl1','mp_treated1','mp_ctrl2','mp_treated2']]
result[result['fulfills_all']==True][['Protein Group Accessions','Protein_Description','diff_prob_rep1','diff_prob_rep2','diff_prob_rep3','pvalue_criteria3']]

# result[result['fulfills_all']==True][['Protein Group Accessions','Protein_Description','diff_rep1','diff_rep2','diff_rep3','same_thermal_shift_direction']]

,Protein Group Accessions,Protein_Description,diff_prob_rep1,diff_prob_rep2,diff_prob_rep3,pvalue_criteria3
104,O43865,Putative adenosylhomocysteinase 2,8.087182e-04,0.000129,0.001089,True
268,P08238,Heat shock protein HSP 90-beta,3.497356e-02,0.030638,0.142328,False
780,Q00796,Sorbitol dehydrogenase,3.272292e-01,0.030638,0.033267,False
873,Q13823,Nucleolar GTP-binding protein 2,1.025302e-02,0.493923,0.017399,False
1004,Q58FF6,Putative heat shock protein HSP 90-beta 4,5.397912e-02,0.049805,0.024801,False
1222,Q8TDG2,Actin-related protein T1,3.275094e-08,0.000004,0.000477,True
1489,Q9BYT8,"Neurolysin, mitochondrial",8.220131e-03,0.000534,0.010275,True
1595,Q9HCG8,Pre-mRNA-splicing factor CWC22 homolog,4.929949e-02,0.030638,0.010564,True
1813,Q9Y3L5,Ras-related protein Rap-2c,1.754186e-08,0.244425,0.044580,False


In [11]:
result.to_csv(path+'result_aggr_end.csv',index=False)